In [48]:
import tp3
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pandas as pd
from faker import Faker
import time 

In [49]:
LONG_A = 100

In [50]:
fake = Faker()
nombres_aleatorios = set()
nombres_aleatorios_list  = []
while len(nombres_aleatorios) < LONG_A:
    nombre = fake.name()
    nombres_aleatorios.add(nombre)

nombres_aleatorios_list = list(nombres_aleatorios)

In [51]:
subsets = []
for i in range(LONG_A): 
    subset = []
    for j in range(i): 
        new_set = set()
        for k in range(random.randint(1, 10)):
            pos = random.randint(1, LONG_A - 1)
            new_set.add(nombres_aleatorios_list[pos])
        subset.append(new_set)
    subsets.append(subset)

In [52]:
times = pd.DataFrame()

In [53]:
def calcular_tiempos(subsets, funcion, a):
    
    time_list = []  # Rename 'time' to avoid conflicts with the time module
    lengths = []    # Use a different name for the list of lengths

    for i in range(len(subsets)):
        lengths.append(i)
        start_time = time.time()
        funcion(subsets[i], a)
        end_time = time.time()
        elapsed_time = end_time - start_time
        time_list.append(elapsed_time)

In [54]:
tiempos_backtracking, len_backtracking = calcular_tiempos(subsets, tp3.search_for_min_hitting_set, nombres_aleatorios)

In [ ]:
print(tiempos_backtracking)